In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
from io import StringIO
m = gp.Model('mip1')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-04


In [2]:
data = 'RF_YN3.csv'
df = pd.read_csv(data)
df.head()

,GWD (m),Elevation,L (km),Slope (%),PGA (g),Displ (m),Target
0,0.370809,0.909116,0.319117,5.465739,0.546270,0.000000,0
1,1.300896,1.123009,0.211770,0.905948,0.532398,0.195432,0
2,1.300896,0.847858,0.195947,0.849104,0.532398,0.217259,0
3,1.788212,2.044325,0.115795,0.451034,0.542307,0.239763,0
4,1.637517,2.003797,0.137265,0.941866,0.545784,0.377358,1


In [3]:
from sklearn.model_selection import train_test_split
#define features matrix (X) and target (y)
X = df.drop(['Displ (m)'], axis = 1)
y = df.drop(['GWD (m)', 'Elevation',  'L (km)', 'Slope (%)','PGA (g)','Displ (m)'],axis=1)

# implementing train-test-split
X_train_target, X_test_target, y_train_df, y_test_df = train_test_split(X, y, test_size=0.2, random_state=0)
X_all = X.drop(['Target'], axis=1)
X_train_df = X_train_target.drop(['Target'], axis = 1)
X_test_df = X_test_target.drop(['Target'], axis = 1)
X_train = X_train_df.values
X_test = X_test_df.values
y_train_df = pd.DataFrame(y_train_df)
y_test_df = pd.DataFrame(y_test_df)
y_train = y_train_df.values
y_test = y_test_df.values

In [4]:
rows, cols = X_train.shape
print(rows, cols)

5832 5


### Determine the depth

In [31]:
depth = 3

## CART

In [32]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth = depth, max_leaf_nodes = 2**depth)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, max_leaf_nodes=8)

In [33]:
total_nodes = clf.tree_.node_count
leaf_nodes = round(total_nodes / 2)
branch_nodes = total_nodes // 2

left_nodes = clf.tree_.children_left
right_nodes = clf.tree_.children_right
left_nodes = left_nodes[0:branch_nodes]
right_nodes = right_nodes[0:branch_nodes]

feature = clf.tree_.feature
initial_a = []
initial_a_tmp = []
for i in range(len(feature)):
    if feature[i] != -2 and feature[i+1] != -2:
        initial_a.append(feature[i])
    elif feature[i] != -2 and feature[i+1] == -2:
        initial_a_tmp.append(feature[i])
    else:
        continue
initial_a.extend(initial_a_tmp)
initial_a

[4, 1, 4, 2, 2, 2, 0]

In [34]:
threshold = clf.tree_.threshold
initial_b = []
initial_b_tmp = []
for i in range(len(threshold)):
    if threshold[i] != -2 and threshold[i+1] != -2:
        initial_b.append(threshold[i])
    elif threshold[i] != -2 and threshold[i+1] == -2:
        initial_b_tmp.append(threshold[i])
    else:
        continue
initial_b.extend(initial_b_tmp)
initial_b

[0.42078618705272675,
 2.318694829940796,
 0.480786070227623,
 0.11024847999215126,
 1.9588893055915833,
 0.6990095376968384,
 1.947537124156952]

In [35]:
clf.score(X_test, y_test)

0.6641535298149417

# OCT
## Determine the alpha and K

In [36]:
alpha = 0.5
K = 2

In [37]:
max_diff = np.max(X_train, 0) - np.min(X_train, 0)
epsilon = np.ones([1, cols], dtype = int) * max_diff
for i in range(cols):
    old = 0
    for j in range(1, rows):
        diff = abs(X_train[j, i] - X_train[old ,i])
        old = j
        if diff < epsilon[0, i] and diff != 0:
            epsilon[0, i] = diff

epsilon = abs(epsilon)
epsilon

array([[3.19004059e-04, 4.25815582e-04, 2.42393191e-04, 1.10328197e-04,
        1.66296959e-05]])

In [38]:
max_epsilon = np.max(epsilon)

n = rows
num_feature = cols

In [39]:
y_train_df = y_train_df.reset_index()
y_train_df.index

RangeIndex(start=0, stop=5832, step=1)

In [40]:
Y = np.zeros([rows,K], dtype = int) - 1

Y[y_train_df.index, y_train_df['Target'].astype(int)] = 1

Y

array([[ 1, -1],
       [ 1, -1],
       [-1,  1],
       ...,
       [ 1, -1],
       [-1,  1],
       [-1,  1]])

In [41]:
l = m.addVars(leaf_nodes, vtype = gp.GRB.BINARY, name = "l")

z = m.addVars(n, leaf_nodes, vtype = gp.GRB.BINARY, name = "z")

N_kt = m.addVars(K, leaf_nodes, vtype = gp.GRB.INTEGER, name = "N_kt")

N_t = m.addVars(leaf_nodes, vtype = gp.GRB.INTEGER, name = "N_t")

c_kt = m.addVars(K, leaf_nodes, vtype = gp.GRB.BINARY, name = "c")

L = m.addVars(leaf_nodes, vtype = gp.GRB.INTEGER, name = "L")

a = m.addVars(branch_nodes, num_feature, vtype = gp.GRB.BINARY, name = 'a')

b = m.addVars(branch_nodes, vtype = gp.GRB.CONTINUOUS, name = "b")

d = m.addVars(branch_nodes, vtype = gp.GRB.BINARY, name = "d")


### Warm start

In [42]:
# warm start using the results of CART algorithm
for i in range(branch_nodes):
    a[i, initial_a[i]].start = 1
    b[i].start = initial_b[i]

m.update()

m.setObjective(L.sum() + alpha * d.sum(), gp.GRB.MINIMIZE)

In [43]:
for i in range(branch_nodes):
    b[i].setAttr(gp.GRB.Attr.LB, 0)
    m.addConstr(b[i] <= d[i])
    m.addConstr(a.sum(i, '*') == d[i])
    m.addConstr(d[i] == 1)
    m.addConstr(l[i] == 1)
    
for i in range(leaf_nodes):
    m.addConstr(L[i] >= 0)
    m.addConstr(N_t[i] == z.sum('*', i))
    m.addConstr(l[i] == c_kt.sum('*', i))
    m.addConstr(z.sum('*', i) >= l[i])
    for j in range(K):
        m.addConstr(L[i] >= N_t[i] - N_kt[j,i] - n * (1 - c_kt[j,i]))
        m.addConstr(L[i] <= N_t[i] - N_kt[j,i] + n * c_kt[j,i])
        m.addConstr(N_kt[j,i] == 1/2 * sum(z.select('*', i) * (Y[:,j] + 1)))

for i in range(n):
    m.addConstr(z.sum(i, '*') == 1)
    
for i in range (leaf_nodes):
    for j in range(n):
        m.addConstr(z[j,i] <= l[i])

all_branch_nodes = list(reversed(range(branch_nodes)))
depth_dict = {}
for i in range(depth):
    depth_dict[i] = sorted(all_branch_nodes[-2**i:])
    for j in range(2**i):
        all_branch_nodes.pop()

all_leaf_nodes = list(range(leaf_nodes))
branch_dict = {}
for i in range(branch_nodes):
    for k in range(depth):
        if i in depth_dict[k]:
            floor_len = len(depth_dict[k])
            step = 2**depth // floor_len
            sliced_leaf = [all_leaf_nodes[i:i+step] for i in range(0, 2**depth, step)]
            idx = depth_dict[k].index(i)
            branch_dict[i] = sliced_leaf[idx]
        else:
            continue
            
for j in range(n):
    for i in range(leaf_nodes):
        for k in range(branch_nodes):
            if i in branch_dict[k]:
                length = len(branch_dict[k])
                idx = branch_dict[k].index(i)
                if idx+1 <= length//2:
                    m.addConstr(sum(a.select(k, '*') * (X_train[j,:] + epsilon[0,:])) <= b[k] + (1 + max_epsilon) * (1 - z[j,i]))
                elif idx+1 > length//2:
                    m.addConstr(sum(a.select(k, '*') * X_train[j,:]) >= b[k] - (1 - z[j,i]))
            else:
                continue

In [44]:
m.Params.timelimit = 1200

Set parameter TimeLimit to value 1200


In [45]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 385128 rows, 93522 columns and 2512148 nonzeros
Model fingerprint: 0xf7135dbf
Variable types: 14 continuous, 93508 integer (93444 binary)
Coefficient statistics:
  Matrix range     [1e-04, 6e+03]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R4 by 1.318694830
Processing MIP start from previous solve: 0 nodes explored in subMIP, total elapsed time 6s
Processing MIP start from previous solve: 0 nodes explored in subMIP, total elapsed time 10s
Processing MIP start from previous solve: 0 nodes explored in subMIP, total elapsed time 15s
Processing MIP start from previous solve: 0 nodes explored in subMIP, total elapsed time 20s
Processing MIP start from previous solve: 0 nodes explored i

In [46]:
for v in m.getVars():
    print(v.varName, v.x)
print('Obj:', m.objVal)

l[0] 1.0
l[1] 1.0
l[2] 1.0
l[3] 1.0
l[4] 1.0
l[5] 1.0
l[6] 1.0
l[7] 1.0
z[0,0] 0.0
z[0,1] 1.0
z[0,2] 0.0
z[0,3] 0.0
z[0,4] 0.0
z[0,5] 0.0
z[0,6] 0.0
z[0,7] 0.0
z[1,0] 0.0
z[1,1] 0.0
z[1,2] 0.0
z[1,3] 0.0
z[1,4] 0.0
z[1,5] 0.0
z[1,6] 1.0
z[1,7] 0.0
z[2,0] 0.0
z[2,1] 0.0
z[2,2] 1.0
z[2,3] 0.0
z[2,4] 0.0
z[2,5] 0.0
z[2,6] 0.0
z[2,7] 0.0
z[3,0] 0.0
z[3,1] 1.0
z[3,2] 0.0
z[3,3] 0.0
z[3,4] 0.0
z[3,5] 0.0
z[3,6] 0.0
z[3,7] 0.0
z[4,0] 0.0
z[4,1] 1.0
z[4,2] 0.0
z[4,3] 0.0
z[4,4] 0.0
z[4,5] 0.0
z[4,6] 0.0
z[4,7] 0.0
z[5,0] 0.0
z[5,1] 1.0
z[5,2] 0.0
z[5,3] 0.0
z[5,4] 0.0
z[5,5] 0.0
z[5,6] 0.0
z[5,7] 0.0
z[6,0] 0.0
z[6,1] 0.0
z[6,2] 1.0
z[6,3] 0.0
z[6,4] 0.0
z[6,5] 0.0
z[6,6] 0.0
z[6,7] 0.0
z[7,0] 0.0
z[7,1] 0.0
z[7,2] 1.0
z[7,3] 0.0
z[7,4] 0.0
z[7,5] 0.0
z[7,6] 0.0
z[7,7] 0.0
z[8,0] 0.0
z[8,1] 0.0
z[8,2] 1.0
z[8,3] 0.0
z[8,4] 0.0
z[8,5] 0.0
z[8,6] 0.0
z[8,7] 0.0
z[9,0] 0.0
z[9,1] 1.0
z[9,2] 0.0
z[9,3] 0.0
z[9,4] 0.0
z[9,5] 0.0
z[9,6] 0.0
z[9,7] 0.0
z[10,0] 0.0
z[10,1] 0.0
z[10,2] 0.0
z[10,3] 0.0


### Tree structure

In [47]:
# Obtain the coefficients of a and b
coff_a = np.zeros([branch_nodes, num_feature], dtype = int)
# coff_b = np.zeros(branch_nodes, dtype = int)
coff_b = np.zeros(branch_nodes)

for i in range(branch_nodes):
    tmp1 = m.getVarByName('b' + '[' + str(i) + ']')
#     coff_b[i] = int(tmp1.x)
    coff_b[i] = tmp1.x
    for j in range(num_feature):
        tmp2 = m.getVarByName('a' + '[' + str(i) + ',' + str (j) + ']')
        coff_a[i,j] = int(tmp2.x)


In [48]:
# Obtain the labels of leaf nodes
labels = np.zeros(leaf_nodes, dtype = int) - 1
coff_c = np.zeros([K, leaf_nodes], dtype = int)

for i in range(K):
    for j in range(leaf_nodes):
        tmp3 = m.getVarByName('c' + '[' + str(i) + ',' + str (j) + ']')
        coff_c[i,j] = int(tmp3.x)

k_idx, t_idx = np.where(coff_c == 1)
# for i in range(leaf_nodes):
for i in range(len(k_idx)):
    labels[t_idx[i]] = k_idx[i]

### Test

In [49]:
t_rows, t_cols = X_test.shape

In [50]:
tmp = np.zeros([t_rows, 1], dtype = int)
Y_predict = np.hstack((np.reshape(y_test_df.values, (t_rows, 1)), tmp))

In [51]:
num_nodes = 0
for i in range(branch_nodes): 
    tmp4 = m.getVarByName('d' + '[' + str(i) + ']')
    num_nodes += int(tmp4.x)

In [52]:
init = np.array([], dtype = int).reshape(0, num_feature)
nodes = {}
for i in range(num_nodes * 2):
    nodes[i] = init

# split
for i in range(t_rows):
    if np.dot(coff_a[0,:], np.transpose(X_test[i,:])) < coff_b[0]:
        nodes[0] = np.vstack([X_test[i,:], nodes[0]])
        if np.dot(coff_a[1,:], np.transpose(X_test[i,:])) < coff_b[1]:
            nodes[2] = np.vstack([X_test[i,:], nodes[2]])
            Y_predict[i,1] = labels[0]
            
        elif np.dot(coff_a[1,:], np.transpose(X_test[i,:])) >= coff_b[1]:
            nodes[3] = np.vstack([X_test[i,:], nodes[3]])
            Y_predict[i,1] = labels[1]
            
    elif np.dot(coff_a[0,:], np.transpose(X_test[i,:])) >= coff_b[0]:
        nodes[1] = np.vstack([X_test[i,:], nodes[1]])
        if np.dot(coff_a[2,:], np.transpose(X_test[i,:])) < coff_b[2]:
            nodes[4] = np.vstack([X_test[i,:], nodes[4]])
            Y_predict[i,1] = labels[2]
            
        elif np.dot(coff_a[2,:], np.transpose(X_test[i,:])) >= coff_b[2]:
            nodes[5] = np.vstack([X_test[i,:], nodes[5]])
            Y_predict[i,1] = labels[3]


In [53]:
for i in range(len(nodes)):
    if i > 1:
        print('node' + ' ' + str(i+2) + ' (predicted label is ' + str(labels[i-2]) + ')')
    else:
        print('node' + ' ' + str(i+2))
    
    print(nodes[i])

node 2
[[1.19256854 4.98534918 1.03383657 0.25330943 0.41867661]
 [2.63488197 5.03210735 1.85047661 0.23928691 0.38669553]
 [1.36328852 3.33014154 1.09019042 0.24357565 0.46274331]
 ...
 [2.03284359 2.18458462 0.18846495 0.35618114 0.452941  ]
 [1.29517341 4.31870699 2.17467513 0.37503383 0.38273013]
 [1.52020788 4.53553152 1.40839613 0.99383557 0.40629542]]
node 3
[[2.95343065 3.94618273 0.73247574 3.01280665 0.48382974]
 [2.4036243  4.88375473 1.01740145 1.96030486 0.49008781]
 [2.78557229 2.08717871 0.91125043 0.60694426 0.50658578]
 ...
 [1.4066143  2.02473521 0.60810234 0.21517485 0.53992623]
 [2.97458601 3.65152097 1.37980082 0.37138587 0.50375223]
 [2.81095171 5.39703369 1.65288653 0.95964682 0.49225309]]
node 4 (predicted label is 1)
[[1.19256854 4.98534918 1.03383657 0.25330943 0.41867661]
 [2.63488197 5.03210735 1.85047661 0.23928691 0.38669553]
 [2.25379825 4.18224859 1.08019103 0.73912317 0.3685199 ]
 ...
 [2.79127979 4.06503963 0.81013919 1.3324368  0.41631794]
 [1.2951734

IndexError: index 8 is out of bounds for axis 0 with size 8

### Test Accuracy

In [54]:
1 - sum(np.minimum(abs(Y_predict[:,1] - Y_predict[:,0]), 1)) / t_rows

0.39821795750514055